# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
plt.style.use("seaborn")

In [4]:
plt.rcParams["figure.figsize"] = (12, 8)

# Chargement du fichier/ spécificité des format excel

In [5]:
df = pd.read_excel("isfcom2017.xlsx")

In [6]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,AVERTISSEMENT,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
fichier = pd.ExcelFile("isfcom2017.xlsx")

In [8]:
fichier.sheet_names

['Avertissement', 'définitif', 'définitif_patrimoine']

In [9]:
df = fichier.parse(sheet_name="définitif", skiprows=1)

# Découverte du jeu de données

In [10]:
df.head()

,Région,Départements,Code commune (INSEE),Commune,nombre de redevables,patrimoine moyen en €,impôt moyen en €
0,AUVERGNE-RHONE-ALPES,AIN,01 053,BOURG-EN-BRESSE,157,2261155,8297
1,AUVERGNE-RHONE-ALPES,AIN,01 283,OYONNAX,88,3236170,12078
2,AUVERGNE-RHONE-ALPES,ALLIER,03 185,MONTLUCON,111,2591793,9721
3,AUVERGNE-RHONE-ALPES,ALLIER,03 190,MOULINS,59,2524509,10026
4,AUVERGNE-RHONE-ALPES,ALLIER,03 310,VICHY,177,2369664,7316


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 7 columns):
Région                   388 non-null object
Départements             388 non-null object
Code commune (INSEE)     388 non-null object
Commune                  388 non-null object
nombre de redevables     388 non-null int64
patrimoine moyen en €    388 non-null int64
impôt moyen en €         388 non-null int64
dtypes: int64(3), object(4)
memory usage: 21.3+ KB


In [12]:
df.columns

Index(['Région', 'Départements', 'Code commune (INSEE)', 'Commune',
       'nombre de redevables', 'patrimoine moyen en €', 'impôt moyen en €'],
      dtype='object')

**Récupérer le nombre de personnes imposables**
- par région
- par département

Puis associer le patrimoine moyen par région/département et l'impot moyen.

In [14]:
df.columns

Index(['Région', 'Départements', 'Code commune (INSEE)', 'Commune',
       'nombre de redevables', 'patrimoine moyen en €', 'impôt moyen en €'],
      dtype='object')

In [19]:
par_dpt = df.groupby(by="Départements").agg("sum").drop(axis=1, labels=df.columns[-2:])
par_dpt.head()

,nombre de redevables
Départements,
AIN,245
AISNE,385
ALLIER,347
ALPES-DE-HAUTE-PROVENCE,127
ALPES-MARITIMES,8478


In [20]:
par_regions = df.groupby(by="Région").agg("sum").drop(axis=1, labels=df.columns[-2:])
par_regions.head()

,nombre de redevables
Région,
AUVERGNE-RHONE-ALPES,12674
BOURGOGNE-FRANCHE-COMTE,2100
BRETAGNE,3200
CENTRE-VAL DE LOIRE,2538
CORSE,395


**Pour obtenir le patrimoine moyen/impot moyen il faut pondérer par le nombre de redevables, on ne peut pas se contenter d'agréger par la moyenne**

In [21]:
df["patrimoine_total"] = df["patrimoine moyen en €"]*df["nombre de redevables"]
df["imposition_totale"] = df["impôt moyen en €"]*df["nombre de redevables"]
df.head()

,Région,Départements,Code commune (INSEE),Commune,nombre de redevables,patrimoine moyen en €,impôt moyen en €,patrimoine_total,imposition_totale
0,AUVERGNE-RHONE-ALPES,AIN,01 053,BOURG-EN-BRESSE,157,2261155,8297,355001335,1302629
1,AUVERGNE-RHONE-ALPES,AIN,01 283,OYONNAX,88,3236170,12078,284782960,1062864
2,AUVERGNE-RHONE-ALPES,ALLIER,03 185,MONTLUCON,111,2591793,9721,287689023,1079031
3,AUVERGNE-RHONE-ALPES,ALLIER,03 190,MOULINS,59,2524509,10026,148946031,591534
4,AUVERGNE-RHONE-ALPES,ALLIER,03 310,VICHY,177,2369664,7316,419430528,1294932


In [24]:
par_dpt["patrimoine_moyen"] = (df.groupby(by="Départements").agg("sum")["patrimoine_total"] / par_dpt["nombre de redevables"]).astype(np.int)
par_dpt["imposition_moyenne"] = (df.groupby(by="Départements").agg("sum")["imposition_totale"] / par_dpt["nombre de redevables"]).astype(np.int)
par_dpt.head()

,nombre de redevables,patrimoine_moyen,imposition_moyenne
Départements,,,
AIN,245,2611364,9655
AISNE,385,2543306,11071
ALLIER,347,2467047,8546
ALPES-DE-HAUTE-PROVENCE,127,2587327,12326
ALPES-MARITIMES,8478,2750880,12193


In [26]:
par_regions["patrimoine_moyen"] = (df.groupby(by="Région").agg("sum")["patrimoine_total"] / par_regions["nombre de redevables"]).astype(np.int)
par_regions["imposition_moyenne"] = (df.groupby(by="Région").agg("sum")["imposition_totale"] / par_regions["nombre de redevables"]).astype(np.int)
par_regions.head()

,nombre de redevables,patrimoine_moyen,imposition_moyenne
Région,,,
AUVERGNE-RHONE-ALPES,12674,2612040,9818
BOURGOGNE-FRANCHE-COMTE,2100,2643354,10097
BRETAGNE,3200,2615544,9645
CENTRE-VAL DE LOIRE,2538,2449088,9816
CORSE,395,2682821,11723
